<a href="https://colab.research.google.com/github/shubh3007/My-App/blob/main/n8n_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Step 1: Authenticate and import libraries
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io
import os
import cv2
from pydub import AudioSegment
import ffmpeg
import tempfile
import shutil

# Step 2: Set folder ID in your Drive where files are stored
FOLDER_ID = "14vNHH0itvmq_DiGVAMPxbS0Ck_ozNq_m"  # your folder
FILES = {
    "image1.jpg": None,
    "image2.jpg": None,
    "image3.jpg": None,
    "audio.mp3": None,
    "output.mp4": None
}

# Step 3: Build Drive service
drive_service = build('drive', 'v3')

# Step 4: Helper to download a file by name from folder
def download_file(file_name, folder_id):
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and name='{file_name}' and trashed=false",
        spaces='drive',
        fields="files(id, name)"
    ).execute()
    items = results.get('files', [])
    if not items:
        raise FileNotFoundError(f"File {file_name} not found in folder.")
    file_id = items[0]['id']
    fh = io.FileIO(file_name, 'wb')
    downloader = MediaIoBaseDownload(fh, drive_service.files().get_media(fileId=file_id))
    done = False
    while not done:
        status, done = downloader.next_chunk()
    print(f"Downloaded {file_name}")
    return file_name

# Step 5: Download images and audio
for fname in ["image1.jpg", "image2.jpg", "image3.jpg", "audio.mp3"]:
    FILES[fname] = download_file(fname, FOLDER_ID)

# Step 6: Create video function
def create_video_with_images_and_audio(image_paths, audio_path, output_path):
    if len(image_paths) != 3:
        raise ValueError("Exactly three images required.")

    # Load audio
    audio = AudioSegment.from_file(audio_path)
    audio_duration_sec = len(audio) / 1000.0
    image_duration_sec = audio_duration_sec / 3

    # Load images
    images = [cv2.imread(p) for p in image_paths]
    height, width, _ = images[0].shape
    for i in range(len(images)):
        if images[i].shape[0] != height or images[i].shape[1] != width:
            images[i] = cv2.resize(images[i], (width, height))

    # Create temp video
    temp_dir = tempfile.mkdtemp()
    temp_video_path = os.path.join(temp_dir, "temp_video.mp4")
    fps = 30
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(temp_video_path, fourcc, fps, (width, height))

    frames_per_image = int(image_duration_sec * fps)
    for img in images:
        for _ in range(frames_per_image):
            video_writer.write(img)
    video_writer.release()

    # Combine with audio using ffmpeg-python
    try:
        video_stream = ffmpeg.input(temp_video_path)
        audio_stream = ffmpeg.input(audio_path)
        ffmpeg.output(video_stream, audio_stream, output_path, vcodec='libx264', acodec='aac', shortest=None, strict='-2').run(overwrite_output=True, quiet=True)
    except Exception as e:
        print(f"FFmpeg failed: {e}")
        shutil.copy2(temp_video_path, output_path)
        print("Warning: Created video without audio.")
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)

    return output_path

# Step 7: Run video creation
output_file = "output.mp4"
create_video_with_images_and_audio(
    ["image1.jpg", "image2.jpg", "image3.jpg"],
    "audio.mp3",
    output_file
)
print(f"Video created: {output_file}")

# Step 8: Upload output back to Drive
file_metadata = {'name': 'output.mp4', 'parents':[FOLDER_ID]}
media = MediaFileUpload(output_file, mimetype='video/mp4', resumable=True)
uploaded = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
print(f"Uploaded output.mp4 to Drive with file ID: {uploaded['id']}")


Downloaded image1.jpg
Downloaded image2.jpg
Downloaded image3.jpg
Downloaded audio.mp3
Video created: output.mp4
Uploaded output.mp4 to Drive with file ID: 1XNK4rePDi87gvCgdlleL9Gq-1d4KLjIW
